In [2]:
import ee

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "F:/Download/thu-87486-d486da3e525f.json"

In [5]:
ee.Initialize()
print(ee.Date('2021-01-01').format().getInfo())

2021-01-01T00:00:00


In [6]:
import datetime
import ee
import csv
import os

ee.Initialize()

In [8]:
start_date = ee.Date.fromYMD(2022, 1, 1)
end_date = start_date.advance(1, 'month')

date_string = end_date.format('YYYY_MM')
filename = 'ssm_{}.csv'.format(date_string.getInfo())

In [9]:
output_path = os.path.join(filename)

In [10]:
soilmoisture = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture")
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

In [11]:
# Filter to a state
karnataka = admin2.filter(ee.Filter.eq('ADM1_NAME', 'Karnataka'))

# Select the ssm band
ssm  = soilmoisture.select('ssm')

filtered = ssm .filter(ee.Filter.date(start_date, end_date))

mean = filtered.mean()

stats = mean.reduceRegions(**{
  'collection': karnataka,
  'reducer': ee.Reducer.mean().setOutputs(['meanssm']),
  'scale': 10000,
  'crs': 'EPSG:32643'
  })

In [12]:
# Select columns to keep and remove geometry to make the result lightweight
# Change column names to match your uploaded shapefile
columns = ['ADM2_NAME', 'meanssm']
exportCollection = stats.select(**{
    'propertySelectors': columns,
    'retainGeometry': False})

features = exportCollection.getInfo()['features']

data = []

for f in features:
    data.append(f['properties'])

field_names = ['ADM2_NAME', 'meanssm']

with open(output_path, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(data)
    print('Success: File written at', output_path)

Success: File written at ssm_2022_02.csv


In [1]:
import datetime

today = datetime.date.today()
print(today)

2023-09-21
